In [11]:
import pandas as pd
import re
import emojis
import regex

In [2]:
def getTokens(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if validSender(message):
        splitMessage = message.split(': ') 
        sender = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        sender=None
    return date, time, sender, message
def validSender(text):
    patterns = [
        '([\w]+):',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([\w]+)[\u263a-\U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, text)
    if result:
        return True
    return False
def startDT(line):
    pattern='^([0-9]+)(/)([0-9]+)(/)([0-9][0-9][0-9][0-9]), ([0-9]+):([0-9][0-9]) (am|pm) -'
    result=re.match(pattern, line)
    if result:
        return True
    else:
        return False

data=[]
with open('C:/Users/mujta/Downloads/chat1.txt', encoding="utf-8") as fp:
    buffer=[]
    date,time,sender=None, None, None
    while True:
        line=fp.readline()
        if not line:
            break
        else:
            line=line.strip()
            if startDT(line):
                 if len(buffer) > 0: 
                     data.append([date, time, sender, ' '.join(buffer)]) 
                 buffer.clear() 
                 date, time, sender, message = getTokens(line) 
                 buffer.append(message) 
            else:
                 buffer.append(line)               
df=pd.DataFrame(data, columns=['Date','Time','Sender','Message'])    
df['Date']=pd.to_datetime(df['Date'])                  
df['Time']=pd.to_datetime(df['Time'])                  
df.info()

In [ ]:
print(len(df.Sender.unique()))
df.Sender.unique()

In [30]:
total_messages=df.shape[0]
media_messages=len(df[df['Message'] == '<Media omitted>'])
emoji_count=0
for mes in df['Message']:
    emoji_count+=emojis.count(mes)
pattern=r'(https?://\S+)'
link_count=0
for mes in df['Message']:
    result=re.match(pattern, mes)
    if result:
        link_count+=1
print("Total Messages: "+str(total_messages), "\nMedia: "+str(media_messages), "\nEmojis: "+str(emoji_count), "\nLinks: "+str(link_count))


Total Messages: 39998 
Media: 2990 
Emojis: 14184 
Links: 465
